In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display, clear_output

def plot_fid(B0=1.0, decay_rate=0.5, time=0.0):
    """
    Plot the macroscopic magnetization and FID signal.
    
    Parameters:
    B0 : float
        Strength of the magnetic field (relative scale)
    decay_rate : float
        Rate of decay for the FID signal
    time : float
        Current time (0-1 represents the duration of visualization)
    time_points : int
        Number of time points to display in the FID
    """
    time_points=50
    # Clear previous plot
    clear_output(wait=True)
    
    # Create figure with custom layout
    fig = plt.figure(figsize=(7, 6))
    # Create a gridspec for more control over subplot sizes
    gs = fig.add_gridspec(2, 3, height_ratios=[2, 1], width_ratios=[1, 0.5,1])
    
    # 3D axis for magnetization (top left)
    ax1 = fig.add_subplot(gs[0, 0], projection='3d')
    # Text box for equations (top right)
    ax_text = fig.add_subplot(gs[0, -1])
    ax_text.axis('off')  # Hide axes
    # 2D axis for FID (bottom, spanning both columns)
    ax2 = fig.add_subplot(gs[1, :])
    
    # Set up the magnetization plot
    ax1.set_xlim(-1.2, 1.2)
    ax1.set_ylim(-1.2, 1.2)
    ax1.set_zlim(-0.2, 1.2)
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title('Macroscopic Magnetization')
    
    # Draw B0 field vector
    ax1.quiver(0, 0, 0, 0, 0, 1.0, color='blue', arrow_length_ratio=0.1, label='B₀')
    
    # Draw detection coil area vector (A)
    ax1.quiver(0, 0, 0, 1.0, 0, 0, color='green', arrow_length_ratio=0.1, label='A (coil)')
    
    # Calculate precession
    omega = 2 * np.pi * B0  # Larmor frequency (scaled)
    current_angle = omega * time
    
    # Magnetization starts at 90° to B0 (in the y-axis)
    # The magnetization decays exponentially
    magnitude = np.exp(-decay_rate * time)
    x_position = magnitude * np.sin(np.pi/2) * np.cos(current_angle)  # sin(90°) = 1
    y_position = magnitude * np.sin(np.pi/2) * np.sin(current_angle)  # Initial position is along y
    z_position = 0  # Magnetization is in the xy plane after 90° pulse
    
    # Draw magnetization vector
    ax1.quiver(0, 0, 0, x_position, y_position, z_position, 
             color='red', arrow_length_ratio=0.1, label='M')
    
    # Draw the circular path for precession
    theta_path = np.linspace(0, 2*np.pi, 40)
    radii = np.linspace(magnitude, 1.0, 10)[::-1]  # Decreasing radii
    
    # Draw precession path with decreasing radii to show decay
    # Use fewer radii to improve performance
    radii = np.linspace(magnitude, 1.0, 4)[::-1]  # Reduced from 10 to 4 points
    
    for r in radii:
        # Use fewer points around the circle
        theta_path = np.linspace(0, 2*np.pi, 20)  # Reduced from 40 to 20 points
        x_path = r * np.cos(theta_path)
        y_path = r * np.sin(theta_path)
        z_path = np.zeros_like(theta_path)
        alpha = 0.1 + 0.4 * (r/1.0)  # Fade out for smaller radii
        ax1.plot(x_path, y_path, z_path, 'b--', alpha=alpha)
    
    # Add legend
    ax1.legend(loc='upper right')
    
    # Calculate FID signal
    # FID is proportional to derivative of the x-component of magnetization
    # When M·A is maximum, derivative is zero; when M·A is zero, derivative is maximum
    t_array = np.linspace(0, 3, time_points)  # Extended time range to show full decay
    
    # Magnetic flux through coil is proportional to cos(omega * t)
    # Derivative of cos is -sin, so FID is proportional to sin(omega * t)
    fid_signal = np.exp(-decay_rate * t_array) * np.sin(omega * t_array)
    
    # Convert the current time to an index in the t_array
    current_index = min(int(time * (time_points - 1) / 3), time_points - 1)  # Adjusted for new time range
    
    # Plot FID signal
    ax2.set_xlim(0, 3)  # Extended time range to show full decay
    ax2.set_ylim(-1.1, 1.1)
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Voltage')
    ax2.set_title('Free Induction Decay (FID)')
    
    # Plot the FID signal as discrete points
    colors = ['black'] * time_points
    colors[current_index] = 'red'
    
    # Plot as bars with adjusted width for the extended time range
    ax2.bar(t_array, fid_signal, width=2.4/time_points, color=colors, alpha=0.7)
    
    # Add annotations
    ax2.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
    ax2.grid(True, alpha=0.3)
    
    # Add formulas to text box
    formula_text = (
        f"$\\omega_0 = \\gamma B_0 = {B0:.1f}$\n\n"
        f"$\\Phi = M \\cdot A = M_x$\n\n"
        f"$V_{{\mathrm{{induced}}}} = -\\frac{{d\\Phi}}{{dt}}$\n\n"
        f"$= -\\frac{{d(M_x)}}{{dt}}$\n\n"
        f"$\\propto e^{{-{decay_rate:.2f}t}} \\sin(\\omega_0 t)$"
    )
    ax_text.text(0.1, 0.5, formula_text, fontsize=12, 
                 bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.5'),
                 verticalalignment='center')
    
    plt.tight_layout()
    plt.show()

# Use widgets.interact to create the interactive visualization
widgets.interact(
    plot_fid,
    B0=(0.2, 2.0, 0.2),        # Controls Larmor frequency
    decay_rate=(0.1, 2.0, 0.1), # Controls decay rate (default 1.0)
    time=(0, 2.99, 0.05),      # Current time (extended range)
);

interactive(children=(FloatSlider(value=1.0, description='B0', max=2.0, min=0.2, step=0.2), FloatSlider(value=…

alright this is pretty much as good as it's gonna get.
I just need to make the lower plot a little smaller and it's perfect.

This is perfect; there's no time to mess with it anymore. it illustrates the concept the way I want it to.


I need to generate the code for the next part now.
For the next part, I will only need to have a plot where the top plot is the free induction decay and the bottom plot is an NMR spectrum.
We'll be able to demonstrate the effect of changing the T1 and T2 parameters, and chemical shift.
That will pretty much be everything that's going on in an NMR experiment.

# Relaxation


In [ ]:
## plot FID above and NMR spectrum below
## We're going to cheat and not do a real FFT.

sigma = 0.0000005
T_2 = 1.0
T_1 = 1.0
chemical_shift_ppm = sigma * 10e6
linewidth = 1/T_2
line_height = 1/T_1

def peak(x,chemical_shift,linewidth,line_height):
    return line_height* np.exp(-(x - chemical_shift)**2/(2 * linewidth**2))

x = np.linspace(0,15,100)
y = peak(x,chemical_shift_ppm,linewidth,line_height)
plt.xlim(0,15)
plt.ylim(0,5)
#invert x axis
plt.plot(x,y)

alright, this gets the point across, doesn't it?

now I just need to marry this to the FID plot from before.

omega = (1-sigma)* 5
time_points=2000
# FID is proportional to derivative of the x-component of magnetization
# When M·A is maximum, derivative is zero; when M·A is zero, derivative is maximum
t_array = np.linspace(0, 10, time_points)  # Extended time range to show full decay

# Magnetic flux through coil is proportional to cos(omega * t)
# Derivative of cos is -sin, so FID is proportional to sin(omega * t)
fid_signal = np.exp(-t_array/T_2) * np.sin(omega * t_array)

# Create figure with custom layout
fig = plt.figure(figsize=(7, 6))
# Create a gridspec for more control over subplot sizes
gs = fig.add_gridspec(2, 3, height_ratios=[2, 1], width_ratios=[1, 0.5,1])

# 3D axis for magnetization (top left)
ax2 = fig.add_subplot(gs[0, 0])
ax2.set_xlim(0, 10)  # Extended time range to show full decay
ax2.set_ylim(-1.1, 1.1)
ax2.set_xlabel('Time')
ax2.set_ylabel('Voltage')
ax2.set_title('Free Induction Decay (FID)')

# Plot the FID signal as discrete points
colors = ['black'] * time_points

# Plot as bars with adjusted width for the extended time range
ax2.bar(t_array, fid_signal, width=max(2.4/time_points,0.01), color=colors, alpha=0.7)

# Add annotations
ax2.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
ax2.grid(True, alpha=0.3)


In [87]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

def plot_nmr_visualization(sigma=-0.0000005, T_2=1.0, T_1=1.0):
    """
    Plot FID and NMR spectrum with adjustable parameters.
    
    Parameters:
    sigma : float
        Shielding constant
    T_2 : float
        Transverse relaxation time (affects peak width)
    T_1 : float
        Longitudinal relaxation time (affects peak intensity)
    """
    # Clear previous output
    clear_output(wait=True)
    
    # Calculate derived parameters
    chemical_shift_ppm = sigma
    linewidth = 1/(T_2 * 10) 
    line_height = 1/T_1
    omega = (1-sigma) * 5
    
    # Create figure with custom layout
    fig = plt.figure(figsize=(10, 8))
    gs = fig.add_gridspec(2, 3, height_ratios=[1, 1], width_ratios=[1, 0.5, 1])
    
    # FID plot (top)
    ax1 = fig.add_subplot(gs[0, :])
    
    # Spectrum plot (bottom)
    ax2 = fig.add_subplot(gs[1, :])
    
    # Generate FID signal
    time_points = 1000
    t_array = np.linspace(0, 10, time_points)
    fid_signal = np.exp(-t_array/T_2) * np.sin(omega * t_array)
    
    # Plot FID
    ax1.bar(t_array, fid_signal, width=max(2.4/time_points, 0.01), color='black', alpha=0.7)
    ax1.set_xlim(0, 10)
    ax1.set_ylim(-1.1, 1.1)
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Voltage')
    ax1.set_title('Free Induction Decay (FID)')
    ax1.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
    ax1.grid(True, alpha=0.3)
    
    # Generate NMR spectrum
    def peak(x, chemical_shift, linewidth, line_height):
        return line_height * np.exp(-(x - chemical_shift)**2/(2 * linewidth**2))
    
    x = np.array([0])
    x = np.append(t_array,np.linspace(chemical_shift_ppm - linewidth * 4, chemical_shift_ppm + linewidth * 4, 50))
    x = np.append(t_array,np.array([15]))
    y = peak(x, chemical_shift_ppm, linewidth, line_height)
    
    # Plot spectrum
    ax2.plot(x, y, 'r-')
    ax2.set_xlim(15, 0)  # Inverted x-axis
    ax2.set_ylim(0, 5)
    ax2.set_xlabel('Chemical Shift (ppm)')
    ax2.set_ylabel('Intensity')
    ax2.set_title('NMR Spectrum')
    ax2.grid(True, alpha=0.3)
    
    # Add text explaining parameters
    text_info = (
        f"Chemical Shift: {chemical_shift_ppm:.2f} ppm\n"
        f"T₁ = {T_1:.2f} (affects peak intensity)\n"
        f"T₂ = {T_2:.2f} (affects peak width)\n\n"
        "T₁ (longitudinal relaxation):\n"
        "↑ T₁ → slower recovery → lower intensity\n"
        "↓ T₁ → faster recovery → higher intensity\n\n"
        "T₂ (transverse relaxation):\n"
        "↑ T₂ → slower decay → sharper peaks\n"
        "↓ T₂ → faster decay → broader peaks"
    )
    
    # Add formula for FID
    formula_text = (
        f"$\\sigma = {sigma:.2f}\\times{{}}10^{{-6}}$\n"
        f"$FID(t) = e^{{-t/T_2}} \\sin(\\omega t)$\n"
        f"$\\omega_0 = \\gamma(1-\\sigma)B_0$ \n"
        f"Chemical shift = $-\\sigma \\cdot 10^6$"
    )
    ax1.text(0.98, 0.95, formula_text, transform=ax1.transAxes,
             fontsize=10, verticalalignment='top', horizontalalignment='right',
             bbox=dict(facecolor='white', alpha=0.7))
    
    plt.tight_layout()
    plt.show()
    
    # Display a note
    print("Note: This visualization uses a simplified model of NMR spectra for educational purposes.")


In [88]:

def interactive_nmr_visualization():
    """
    Create interactive widgets for NMR visualization.
    """
    # Create interactive output
    widgets.interact(
        plot_nmr_visualization,
        sigma=(0.1,15,0.1),
        T_2=(0.1,2.0,0.1),   
        T_1=(0.1,2.0,0.1),
    )

# Run the interactive visualization
interactive_nmr_visualization()

interactive(children=(FloatSlider(value=0.1, description='sigma', max=15.0, min=0.1), FloatSlider(value=1.0, d…